# Olal: Decentralized cryptocurrency

Bring the control of finance back to people.

### Don't Trust. Verify.

### When user runs the same account on two or more different machines. Multiple pending transactions are observed, how to handle the book keeeping?

Each machine may observe the pending transactions in different order and signed them in different orders, or even on different receiving book's levels.

To solve this problem. Olal transaction uses two handshake mechanism. Sender sends a pending transaction.

- Case 1: same transaction being signed by two differente machines with the same earn_floor, this is OK.
- Case 2: same transaction being signed by two different machiens with different earn_floors: This means one of the machine's earn stack is not synced. The machine with lower earn_floor should ask the network for the conflict trasanction that already occupies the earn_floor
 this transaction might be signed by the receiver on different machines. But sender only picks one for confirmation transaction. Once sender pickes one confirmation transaction, he signed it to form a finalized transaction

A sumr transaction is a 4 phases payment. They are __pending__, __signed__, __finalize__ and __close__.

- Sender first initiates a payment by broadcasting a phase 1 __pending__ transaction to the network. Such broadcast makes the entire network to turn sender's the expense journal to a locked status. Such mechanism prevents sender from spending more than his income journal's value, it also prevents expense journal record inconsistency due to cancelled transactions in lower levels. We'll discuss more details in the "Journal balance".
- Receiver receives the __pending__ transaction from one or more nodes in the network, verifies the transaction data and confirming sender's income journal has enough funding for this transaction. Receiver also tries to get this __pending__ transaction from as many nodes as possible in the network and verify that there's no two or more different __pending__ transactions on the network from the same sender. After verification, receiver signs the __pending__ transaction and broadcast it to the network, thus changing the transaction from __pending__ to __signed__. 

    All nodes can verify __pending__ transactions and mark it as __invalid__ and discard it in the future.

    If nodes are all honest and the __pending__ transaction is valid, receiver should be able to get a copy the the __pending__ transaction from any nodes on the network. However, dishonest nodes could exist. If some nodes are dishonest and they accepts an invalid __pending__ transaction as valid and provided to the receiver. That's all fine, because any node is only-trust-myself node, receiver can verify the __pending__ transaction by itself.

First phase is pending phase initiated by sender and broadcasted to sumr network. The format of the pending phase is $[c, a, v_{o_1}, \hat{a}_1, v_{o_2}, \hat{a}_2, ..., v_{o_c}, \hat{a}_c, v_{oc}, l_i, l_o]$.

- $v_o$ is the sending value


- What if receiver never accepts a payment? How can the sender cancel the pending transaction?
